In [94]:
# Importando pacotes necessários
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.model_selection import KFold, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

from scipy.stats import spearmanr





In [5]:
## Utilizar esse caso esteja usando o Google Colab

#from google.colab import drive
#drive.mount('/content/drive')

#Carregar arquivo do drive
#df = pd.read_csv('/content/drive/MyDrive/Períodos/2024-2/Aprendizado de Máquina/Projeto/dados.csv')

In [ ]:
# Carregar o arquivo 'dados.csv' no dataframe
df = pd.read_csv('dados.csv')

# Exibir as primeiras linhas do dataframe para verificar se foi carregado corretamente
print(df.head())

In [ ]:
# Criar um dicionário de mapeamento
classe_map = {1: 'A', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2', 6: 'DE'}

# Substituir os valores da coluna 'CLASSE'
df['CLASSE'] = df['CLASSE'].map(classe_map)

# Verificar a alteração
print(df['CLASSE'].unique())

In [ ]:
contagem = df['CLASSE'].value_counts()
print(contagem)


Aplicando o PCA

In [104]:
# Separar as variáveis independentes (X) e a variável dependente (y)
dados_sem_classe = df.drop('CLASSE', axis=1)
classe_1 = df['CLASSE']

In [105]:
# Padronizar os dados antes de aplicar o PCA
scaler = StandardScaler()
dadosPCA_scaled = scaler.fit_transform(dados_sem_classe)

In [106]:
# Aplicar PCA
pca = PCA()
dadosPCA_reduced = pca.fit_transform(dadosPCA_scaled)

In [8]:
# Variância explicada por cada componente
explained_variance = pca.explained_variance_ratio_

In [9]:
# Variância acumulada
cumulative_variance = np.cumsum(explained_variance)

In [ ]:
# Plotar a variância explicada e a acumulada em gráficos de barras
plt.figure(figsize=(10, 6))

# Gráfico de barras para a variância explicada
plt.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.7, label='Variância Explicada (%)')
plt.title('Análise de Variância com PCA')
plt.xlabel('Número de Componentes Principais')
plt.ylabel('Porcentagem de Variância (%)')
plt.legend(loc='best')
plt.grid(True)
plt.show()


In [ ]:
# Encontrar o número de componentes que explicam
n_components = np.argmax(cumulative_variance >= 0.8) + 1

print(f"Número de componentes principais: {n_components}")

In [ ]:
# Gráfico de linha para a variância acumulada
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', color='red', label='Variância Acumulada (%)')
plt.title('Análise de Variância com PCA')
plt.xlabel('Número de Componentes Principais')
plt.ylabel('Porcentagem de Variância (%)')

plt.axhline(y=0.70, color='blue', linestyle='--', linewidth=1.5, label='70%')
plt.axhline(y=0.80, color='green', linestyle='--', linewidth=1.5, label='80%')
plt.axhline(y=0.90, color='red', linestyle='--', linewidth=1.5, label='90%')


plt.axvline(x=n_components, color='purple', linestyle='--', linewidth=1.5, label='Quantidade de Componentes')


plt.legend(loc='best')
plt.grid(True)
plt.show()

In [13]:
# Aplicar PCA com o número de componentes selecionados
pca = PCA(n_components=n_components)
dadosPCA_reduced = pca.fit_transform(dadosPCA_scaled)

In [14]:
# Criar um DataFrame com as componentes principais
df_pca = pd.DataFrame(dadosPCA_reduced, columns=[f'PC{i+1}' for i in range(n_components)])


In [15]:
# Adicionar a coluna da variável alvo
df_pca['CLASSE'] = classe_1.reset_index(drop=True)


In [ ]:
#Dataframe criado com as componentes principais

df_pca.head()

O PCA selecionou 38 componentes para explicar 80% dos dados. Será criado outro dataframe que seleciona as variáveis mais correlatadas com a primeira componente principal.

In [17]:
#Correlacao Selecionada, acima de:

corre = 0.4

In [18]:
# Ajustar o PCA aos dados e calcular as componentes principais
pca_components = dadosPCA_reduced.copy()

In [ ]:
# Calcular a correlação entre as variáveis originais e a primeira componente principal pela correlação de spearman
corr_with_PC1 = dados_sem_classe.apply(lambda x: x.corr(pd.Series(pca_components[:, 0]), method='spearman'))

# Ordenar as variáveis por correlação absoluta com a primeira componente principal
corr_with_PC1_sorted = corr_with_PC1.abs().sort_values(ascending=False)

# Criar um gráfico de barras para visualizar as correlações
plt.figure(figsize=(10, 6))
sns.barplot(x=corr_with_PC1_sorted.index, y=corr_with_PC1_sorted.values)
plt.xticks(rotation=90)
plt.xlabel('Variáveis')
plt.ylabel('Correlação com PC1')
plt.title('Correlação das Variáveis com a Primeira Componente Principal (PC1)')
plt.axhline(y=corre, color='red', linestyle='--', linewidth=1.5)
plt.tight_layout()
plt.show()

In [ ]:
# Calcular a correlação entre as variáveis originais e a primeira componente principal pela correlação de Pearson
corr2_with_PC1 = dados_sem_classe.apply(lambda x: x.corr(pd.Series(pca_components[:, 0]), method='pearson'))

# Ordenar as variáveis por correlação absoluta com a primeira componente principal
corr2_with_PC1_sorted = corr2_with_PC1.abs().sort_values(ascending=False)

# Criar um gráfico de barras para visualizar as correlações
plt.figure(figsize=(10, 6))
sns.barplot(x=corr2_with_PC1_sorted.index, y=corr_with_PC1_sorted.values)
plt.xticks(rotation=90)
plt.xlabel('Variáveis')
plt.ylabel('Correlação com PC1')
plt.title('Correlação das Variáveis com a Primeira Componente Principal (PC1)')
plt.axhline(y=corre, color='red', linestyle='--', linewidth=1.5)
plt.tight_layout()
plt.show()

In [21]:
# Filtrar variáveis com maior correlação
selected_vars = corr_with_PC1_sorted[corr_with_PC1_sorted > corre].index

In [22]:
# Criar um novo dataframe com as variáveis selecionadas
df_selected = dados_sem_classe[selected_vars].copy()

# Adicionar a coluna CLASSE ao novo dataframe
df_selected['CLASSE'] = classe_1.reset_index(drop=True)

In [ ]:
#Novo dataframe com as variaveis mais correlatadas com a primeira componente principal

df_selected.head()
#print(df_selected.shape)

Aplicando redução pelo coeficiente de spearman

In [58]:
# Calcular a correlação de Spearman para as variáveis
spearman_corr = dados_sem_classe.corr(method='spearman')

In [59]:
# Definir um limiar (threshold) para selecionar as variáveis com alta correlação
threshold = 0.3

In [60]:
# Selecionar as variáveis mais correlacionadas
correlated_vars = np.where(np.abs(spearman_corr) > threshold)
correlated_vars = [(spearman_corr.index[x], spearman_corr.columns[y]) for x, y in zip(*correlated_vars) if x != y]

In [ ]:
# Exibir a matriz de correlação
spearman_corr

In [ ]:
# Plotar o heatmap da matriz de correlação
plt.figure(figsize=(12, 8))
sns.heatmap(spearman_corr, annot=False, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Matriz de Correlação (Spearman)')
plt.show()

In [63]:
# Remover duplicatas e manter uma cópia da variável apenas
selected_columns = list(set([var[0] for var in correlated_vars]))

In [64]:
# Criar um novo dataframe com essas variáveis correlacionadas
df_spearman = dados_sem_classe[selected_columns].copy()


In [65]:
# Adicionar a coluna da variável alvo
df_spearman['CLASSE'] = classe_1.reset_index(drop=True)


In [ ]:
# Exibir as primeiras linhas do novo DataFrame
df_spearman.head()

O algoritmo não encontrou variáveis correlatadas com mais de 70%, por isso será criado um dataframe com as variáveis que possuem maior correlação com as variáveis de saída.

In [139]:
#Correlação a ser selecionada

cor = 0.15

In [140]:
# Converter a variável categórica 'classe_1' para valores numéricos
classe_1_numerica = pd.factorize(classe_1)[0]


In [141]:
# Calcular a correlação de Spearman entre as variáveis de entrada e a variável de saída numérica
spearman_corr_with_target = dados_sem_classe.apply(lambda x: x.corr(pd.Series(classe_1_numerica), method='spearman'))

In [142]:
# Ordenar as variáveis por ordem de correlação (absoluta) com a variável de saída
spearman_corr_with_target = spearman_corr_with_target.abs().sort_values(ascending=False)

In [ ]:
# Exibir a correlação de cada variável de entrada com a variável de saída
print(spearman_corr_with_target)


In [144]:
# Filtrar as variáveis que têm correlação maior que 0.5 em valor absoluto
selected_features = spearman_corr_with_target[spearman_corr_with_target > cor].index

In [145]:
# Criar um novo DataFrame apenas com essas variáveis
df_high_corr = dados_sem_classe[selected_features].copy()

In [146]:
# Adicionar a coluna da variável alvo
df_high_corr['CLASSE'] = classe_1.reset_index(drop=True)

In [ ]:
# Exibir as primeiras linhas do novo DataFrame
print(df_high_corr.head(10))

In [ ]:
df_high_corr

DataFrames criados até aqui:

df_pca - Componentes principais;
df_selected - Variaveis com maior correlacao com a Primeira Componente Principal;
df_spearman - Remoção das variáveis mais correlatadas;
df_high_corr - Variaveis de entrada com maior correlação com as variáveis de saída;

Aplicando a seleção de Dados por k-fold

In [150]:
#Selecionando o dataframe

df_kfold = df_pca.copy()
#df_kfold = df_selected.copy()
#df_kfold = df_spearman.copy()
#df_kfold = df_high_corr.copy()

In [151]:
# Separando as variáveis objetivo
X = df_kfold.drop('CLASSE', axis=1)
y = df_kfold['CLASSE']


In [152]:
#Definindo o número de folds

num_folds = 7
sub_num_folds = 7

In [69]:
#Configurar o k-Fold Principal

kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

In [70]:
#Dividir os Dados para o k-fold teste

fold_indices = list(kf.split(X, y))

In [74]:
# Iterar sobre cada fold principal
for fold, (train_index, test_index) in enumerate(fold_indices):
    # Dados de treino e teste para o fold principal
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Aplicar o sub-k-fold para treino e validação
    sub_kf = StratifiedKFold(n_splits=sub_num_folds, shuffle=True, random_state=42)

    # Armazenar os índices dos folds secundários
    sub_fold_indices = list(sub_kf.split(X_train, y_train))

    for sub_fold, (sub_train_index, sub_val_index) in enumerate(sub_fold_indices):
        # Dados de treino e validação para o sub-k-fold
        X_sub_train, X_sub_val = X_train.iloc[sub_train_index], X_train.iloc[sub_val_index]
        y_sub_train, y_sub_val = y_train.iloc[sub_train_index], y_train.iloc[sub_val_index]

        # Aqui você pode treinar e validar seu modelo
        # Exemplo: print(f'Fold {fold + 1}, Sub-Fold {sub_fold + 1}')


Testando disposição das Classes

In [ ]:
from collections import Counter
import pandas as pd

# Exemplo de dataframe com rótulos (alvo)
y = pd.Series(y)  # Supondo que seus rótulos estejam em um array ou lista chamada 'y'

# Inicializar o StratifiedKFold
kfold = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)

# Iterar pelos folds principais
for fold, (train_index, test_index) in enumerate(kfold.split(X, y)):
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Contagem das classes no treino e teste
    train_counts = Counter(y_train)
    test_counts = Counter(y_test)
    
    print(f"Fold {fold + 1}")
    print(f"Distribuição no treino: {train_counts}")
    print(f"Distribuição no teste: {test_counts}\n")


In [ ]:
from collections import Counter
import pandas as pd
from sklearn.model_selection import StratifiedKFold

# Suponha que X seja um DataFrame e y seja uma Series
# Se X é um DataFrame e y é uma Series
kfold = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)

# Iterar pelos folds principais
for fold, (train_index, test_index) in enumerate(kfold.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Contagem das classes no treino e teste
    train_counts = Counter(y_train)
    test_counts = Counter(y_test)
    
    print(f"Fold {fold + 1}")
    print(f"Distribuição no treino: {train_counts}")
    print(f"Distribuição no teste: {test_counts}\n")

    # Subdividir o treino em subfolds
    sub_kfold = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)
    
    for sub_fold, (sub_train_index, sub_val_index) in enumerate(sub_kfold.split(X_train, y_train)):
        y_sub_train = y_train.iloc[sub_train_index]
        y_sub_val = y_train.iloc[sub_val_index]
        
        # Contagem das classes nos subfolds
        sub_train_counts = Counter(y_sub_train)
        sub_val_counts = Counter(y_sub_val)
        
        print(f"  Subfold {sub_fold + 1}")
        print(f"  Distribuição no treino: {sub_train_counts}")
        print(f"  Distribuição na validação: {sub_val_counts}\n")



Aplicando as redes neurais no k-fold

In [85]:
# Função para criar o modelo de rede neural
def create_model(input_shape, num_classes):
    model = Sequential([
        Dense(12, activation='relu', input_shape=(input_shape,)),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [86]:
# Lista para armazenar as métricas de avaliação
validation_reports = []

In [87]:
best_model_path = 'best_model.keras'  # Local para salvar o melhor modelo

# Configurar o callback para salvar o melhor modelo com base na métrica de validação
checkpoint = ModelCheckpoint(best_model_path, monitor='val_accuracy', save_best_only=True, mode='max')

# Variável para armazenar o melhor desempenho
best_val_accuracy = 0
best_fold = None

In [93]:
# Inicializar uma lista para armazenar a precisão ponderada de cada fold
fold_accuracies = []

In [ ]:
# Codificar os rótulos para garantir que estejam no intervalo esperado
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Aplicar k-Fold e Sub-K-Fold
for fold, (train_index, test_index) in enumerate(fold_indices):
    # Dados de treino e teste para o fold principal
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]
    
    # Aplicar Sub-K-Fold para treino e validação
    sub_fold_indices = list(sub_kf.split(X_train, y_train))
    
    for sub_fold, (sub_train_index, sub_val_index) in enumerate(sub_fold_indices):
        # Dados de treino e validação para o sub-k-fold
        X_sub_train, X_sub_val = X_train.iloc[sub_train_index], X_train.iloc[sub_val_index]
        y_sub_train, y_sub_val = y_train[sub_train_index], y_train[sub_val_index]
        
        # Criar o modelo
        num_classes = len(label_encoder.classes_)
        model = create_model(X.shape[1], num_classes)
        
        # Treinar o modelo e salvar o histórico
        history = model.fit(
            X_sub_train, y_sub_train, 
            epochs=100, batch_size=32, 
            verbose=0, validation_data=(X_sub_val, y_sub_val),
            callbacks=[checkpoint]
        )

        # Avaliar no conjunto de teste
        y_test_pred = model.predict(X_test)
        y_test_pred_classes = np.argmax(y_test_pred, axis=1)
        
        # Calcular acurácia de teste
        test_accuracy = np.mean(y_test_pred_classes == y_test)

        # Adicionar a precisão ao dicionário de precisões dos folds
        fold_accuracies.append({
            'fold': fold,
            'sub_fold': sub_fold,
            'test_accuracy': test_accuracy
        })

        # Atualizar o melhor modelo se este fold for melhor
        if test_accuracy > best_val_accuracy:
            best_val_accuracy = test_accuracy
            best_fold = (fold, sub_fold)
        
        # # Plotar o gráfico de perda (loss) e acurácia (accuracy)
        # plt.figure(figsize=(12, 5))

        # # Perda (loss)
        # plt.subplot(1, 2, 1)
        # plt.plot(history.history['loss'], label='Train Loss')
        # plt.plot(history.history['val_loss'], label='Validation Loss')
        # plt.title(f'Fold {fold + 1} Sub-Fold {sub_fold + 1} - Loss')
        # plt.xlabel('Epochs')
        # plt.ylabel('Loss')
        # plt.legend()

        # # Acurácia (accuracy)
        # plt.subplot(1, 2, 2)
        # plt.plot(history.history['accuracy'], label='Train Accuracy')
        # plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        # plt.title(f'Fold {fold + 1} Sub-Fold {sub_fold + 1} - Accuracy')
        # plt.xlabel('Epochs')
        # plt.ylabel('Accuracy')
        # plt.legend()

        # # Mostrar o gráfico
        # plt.show()


In [ ]:
# Exibir precisões ponderadas de cada fold
for accuracy in fold_accuracies:
    print(f"Fold {accuracy['fold']}, Sub-Fold {accuracy['sub_fold']}: Acurácia de Teste = {accuracy['test_accuracy']:.4f}")

In [ ]:
# Carregar o melhor modelo
best_model = load_model(best_model_path)

# Fazer previsões no conjunto de teste final
y_test_pred = best_model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)

# Gerar a matriz de confusão
cm = confusion_matrix(y_test, y_test_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

# Exibir a acurácia final do melhor modelo
print(f"Melhor acurácia no fold {best_fold[0]+1}, sub-fold {best_fold[1]+1}: {best_val_accuracy}")

In [ ]:
# Calcular a matriz de confusão original
conf_matrix = confusion_matrix(y_test, y_test_pred_classes)

# Frequência das classes no conjunto de dados
class_counts = np.bincount(y_test)  # Conta o número de ocorrências de cada classe
class_weights = 1.0 / class_counts  # Calcula o peso de cada classe

# Matriz de confusão ponderada
weighted_conf_matrix = conf_matrix * class_weights[:, np.newaxis]

# Exibir a matriz de confusão ponderada
print("Matriz de Confusão Ponderada:")
print(weighted_conf_matrix)

# Calcular a precisão de cada classe
precisions = []
for i in range(weighted_conf_matrix.shape[0]):
    true_positives = weighted_conf_matrix[i, i]  # Verdadeiros Positivos
    false_positives = sum(weighted_conf_matrix[:, i]) - true_positives  # Falsos Positivos
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    precisions.append(precision)

# Exibir a precisão de cada classe
for idx, precision in enumerate(precisions):
    print(f'Precisão da Classe {idx}: {precision:.4f}')


In [ ]:
from sklearn.metrics import confusion_matrix, precision_score

# Armazenar as precisões por classe para cada sub-fold
sub_fold_precisions = []

# Exibir a precisão de cada classe
for fold, (train_index, test_index) in enumerate(fold_indices):
    # Dados de treino e teste para o fold principal
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]
    
    # Aplicar Sub-K-Fold para treino e validação
    sub_fold_indices = list(sub_kf.split(X_train, y_train))
    
    for sub_fold, (sub_train_index, sub_val_index) in enumerate(sub_fold_indices):
        # Dados de treino e validação para o sub-k-fold
        X_sub_train, X_sub_val = X_train.iloc[sub_train_index], X_train.iloc[sub_val_index]
        y_sub_train, y_sub_val = y_train[sub_train_index], y_train[sub_val_index]
        
        # Criar o modelo
        num_classes = len(label_encoder.classes_)
        model = create_model(X.shape[1], num_classes)
        
        # Treinar o modelo e salvar o histórico
        history = model.fit(
            X_sub_train, y_sub_train, 
            epochs=100, batch_size=32, 
            verbose=0, validation_data=(X_sub_val, y_sub_val),
            callbacks=[checkpoint]
        )

        # Avaliar no conjunto de teste
        y_test_pred = model.predict(X_test)
        y_test_pred_classes = np.argmax(y_test_pred, axis=1)
        
        # Calcular a matriz de confusão
        cm = confusion_matrix(y_test, y_test_pred_classes, labels=np.arange(num_classes))
        
        # Calcular a precisão de cada classe
        class_precisions = precision_score(y_test, y_test_pred_classes, average=None)
        
        # Armazenar a precisão de cada classe para o sub-fold
        sub_fold_precisions.append({
            'fold': fold,
            'sub_fold': sub_fold,
            'precisions': class_precisions
        })

In [ ]:
# Exibir precisões de cada classe para cada sub-fold
for sub_fold_precision in sub_fold_precisions:
    print(f"Fold {sub_fold_precision['fold']}, Sub-Fold {sub_fold_precision['sub_fold']}:")
    for i, precision in enumerate(sub_fold_precision['precisions']):
        print(f"  Precisão da Classe {i}: {precision:.4f}")